In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer, PolynomialFeatures, KBinsDiscretizer, SplineTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
import sys, os
from utils import get_data_path
from utils import get_save_path

# 1. Load the dataset

In [2]:
df = pd.read_csv(get_data_path("cell2celltrain.csv"))

# Convert target column to binary

In [3]:
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

# 2. Feature Engineering and Cleaning

# Drop highly redundant or uninformative columns

In [4]:
#check if 'UniqueSubs'and 'ActiveSubs' are highly correlated
corr = df[['UniqueSubs','ActiveSubs']].corr().iloc[0,1]
print("UniqueSubs vs ActiveSubs corr =", corr)

UniqueSubs vs ActiveSubs corr = 0.7757755146495737


# Create 'InactiveSubs' feature

In [5]:
# InactiveSubs = UniqueSubs - ActiveSubs
df['InactiveSubs'] = df['UniqueSubs'] - df['ActiveSubs']

In [6]:
#highly corelated columns
df['HandsetDiff'] = df['Handsets'] - df['HandsetModels']

print("Difference counts between Handsets and HandsetModels:")
print(df['HandsetDiff'].value_counts().sort_index())

same_ratio = (df['HandsetDiff'] == 0).mean()
print(f"\nPercentage of rows where Handsets == HandsetModels: {same_ratio:.2%}")


Difference counts between Handsets and HandsetModels:
HandsetDiff
0.0     42166
1.0      6620
2.0      1473
3.0       467
4.0       166
5.0        75
6.0        35
7.0        15
8.0        13
9.0         3
10.0        5
11.0        5
12.0        1
15.0        1
19.0        1
Name: count, dtype: int64

Percentage of rows where Handsets == HandsetModels: 82.60%


In [7]:
# delete columns that only have one value
const_cols = [col for col in df.columns if df[col].nunique(dropna=False) == 1]

print("Columns with constant value (only one unique value):")
print(const_cols)
df = df.drop(columns=const_cols)

# numeric value close to 0 and mean around 0
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

low_mean_cols = [col for col in numeric_cols if df[col].mean() < 0.05]

print("\nNumeric columns with mean close to 0 (possibly uninformative):")
print(low_mean_cols)

# delete CallForwardingCalls based on the eda that CallForwardingCalls has 0 correlation with churn


Columns with constant value (only one unique value):
[]

Numeric columns with mean close to 0 (possibly uninformative):
['PercChangeMinutes', 'PercChangeRevenues', 'CallForwardingCalls', 'RetentionCalls', 'RetentionOffersAccepted']


# Drop Uneccessary Columns

In [8]:
# 刪除與 InactiveSubs 高度重複的欄位
df = df.drop(columns=['UniqueSubs', 'ActiveSubs'])

# 刪除 BlockedCalls 與 DroppedBlockedCalls（保留 DroppedCalls）
df = df.drop(columns=['BlockedCalls', 'DroppedBlockedCalls'])

# 刪除與 Handsets 高度重複的 HandsetModels
df = df.drop(columns=['HandsetModels'])

# 刪除與 Churn 無相關的 CallForwardingCalls
df = df.drop(columns=['CallForwardingCalls'])

# 刪除互補 dummy
df = df.drop(columns=['NotNewCellphoneUser'])  
# 刪除無意義 ID 欄位
df = df.drop(columns=['CustomerID'])  

# 3. Remove Duplicate Rows

In [9]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

# 4. Train/Validation/Test Split

In [10]:
X = df.drop(columns=['Churn'])
y = df['Churn']
# test set（20%）
X_main, X_test, y_main, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# train set 60%
X_train, X_val, y_train, y_val = train_test_split(
    X_main, y_main, test_size=0.25, random_state=42, stratify=y_main
)

# 5. Custom Target Mean Encoder (with smoothing)

In [11]:
# Target Mean Encoder
class TargetMeanEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col, target='Churn', smoothing=10):
        self.col = col
        self.target = target
        self.smoothing = smoothing

    def fit(self, X, y):
        df = pd.DataFrame({self.col: X[self.col], self.target: y})
        self.global_mean_ = df[self.target].mean()
        agg = df.groupby(self.col)[self.target].agg(['mean', 'count'])
        smoothing = 1 / (1 + np.exp(-(agg['count'] - self.smoothing))) # sigmoid smoothing
        self.mapping_ = self.global_mean_ * (1 - smoothing) + agg['mean']
        return self

    def transform(self, X):
        return X[[self.col]].apply(lambda col: col.map(self.mapping_).fillna(self.global_mean_))



# 6. Special Feature Preprocessor： HandsetPrice & Outlier（invalid value-> NaN）

In [12]:
# Check negetive values in those columns
non_negative_cols = [
    'MonthlyRevenue', 'MonthlyMinutes', 'OverageMinutes',
    'TotalRecurringCharge', 'DirectorAssistedCalls', 'RoamingCalls',
    'DroppedCalls', 'UnansweredCalls', 'CustomerCareCalls',
    'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
    'PeakCallsInOut', 'OffPeakCallsInOut', 'CallWaitingCalls',
    'Handsets', 'CurrentEquipmentDays', 'RetentionCalls',
    'RetentionOffersAccepted', 'ReferralsMadeBySubscriber',
    'AdjustmentsToCreditRating', 'InactiveSubs'
]

for col in non_negative_cols:
    num_neg = (X_train[col] < 0).sum()
    if num_neg > 0:
        print(f"{col} has {num_neg} negative values")


MonthlyRevenue has 1 negative values
TotalRecurringCharge has 6 negative values
CurrentEquipmentDays has 45 negative values


In [13]:
def preprocess_special_features(df):
    df = df.copy()
    # HandsetPrice
    df['HandsetPrice_Clean'] = pd.to_numeric(df['HandsetPrice'], errors='coerce')
    df['HandsetPrice_Unknown'] = (df['HandsetPrice'] == 'Unknown').astype(int)
    df.drop(columns=['HandsetPrice'], inplace=True)  

    
    non_negative_cols = [
        'MonthlyRevenue', 'MonthlyMinutes', 'OverageMinutes',
        'TotalRecurringCharge', 'DirectorAssistedCalls', 'RoamingCalls',
        'DroppedCalls', 'UnansweredCalls', 'CustomerCareCalls',
        'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
        'PeakCallsInOut', 'OffPeakCallsInOut', 'CallWaitingCalls',
        'Handsets', 'CurrentEquipmentDays', 'RetentionCalls',
        'RetentionOffersAccepted', 'ReferralsMadeBySubscriber',
        'AdjustmentsToCreditRating', 'InactiveSubs'
    ]
    
    for col in non_negative_cols:
        df[col] = df[col].mask(df[col] < 0, np.nan)

    return df

# sklearn-compatible transformer
special_feature_processor = FunctionTransformer(preprocess_special_features)

# 7. Column Selector(deal with missing value)

In [14]:
class ColumnClassifier(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.numeric_cols_ = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
        self.categorical_cols_ = X.select_dtypes(include=['object', 'category']).columns.tolist()
        if 'HandsetPrice_Clean' in X.columns:
            self.numeric_cols_.append('HandsetPrice_Clean')
        if 'HandsetPrice_Unknown' in X.columns:
            self.categorical_cols_.append('HandsetPrice_Unknown')
        self.ordinal_cols_ = ['CreditRating', 'IncomeGroup']
        self.nominal_cols_ = [col for col in self.categorical_cols_ if col not in self.ordinal_cols_ and col != 'ServiceArea']
        return self
    def transform(self, X): 
        return X

# 8. Build pipeline constructor

In [15]:
def build_pipeline(use_scaling=True):
    def _pipeline(X):
        X = special_feature_processor.fit_transform(X)
        selector = ColumnClassifier().fit(X)

        numeric_steps = [('imputer', SimpleImputer(strategy='median'))]
        if use_scaling:
            numeric_steps.append(('scaler', StandardScaler()))
        numeric_pipe = Pipeline(numeric_steps)

        ordinal_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder())
        ])

        nominal_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
        ])

        return Pipeline([
            ('special', special_feature_processor),
            ('col_transform', ColumnTransformer([
                ('num', numeric_pipe, selector.numeric_cols_),
                ('ord', ordinal_pipe, selector.ordinal_cols_),
                ('nom', nominal_pipe, selector.nominal_cols_),
                ('service', TargetMeanEncoder(col='ServiceArea'), ['ServiceArea'])
            ]))
        ])
    return _pipeline


# 9. Apply Preprocessing Pipelines

In [16]:
baseline_builder = build_pipeline(use_scaling=True)
deep_builder = build_pipeline(use_scaling=True)

baseline_pipeline = baseline_builder(X_train)
deep_pipeline = deep_builder(X_train)

X_train_base = baseline_pipeline.fit_transform(X_train, y_train)
X_val_base = baseline_pipeline.transform(X_val)
X_test_base = baseline_pipeline.transform(X_test)

X_train_deep = deep_pipeline.fit_transform(X_train, y_train)
X_val_deep = deep_pipeline.transform(X_val)
X_test_deep = deep_pipeline.transform(X_test)

# Save pipeline objects
joblib.dump(baseline_pipeline, get_save_path("baseline.pkl"))
joblib.dump(deep_pipeline, get_save_path("deep.pkl"))

# Save processed datasets
pd.DataFrame(X_train_base).to_csv(get_save_path("X_train_base.csv"), index=False)
pd.DataFrame(X_val_base).to_csv(get_save_path("X_val_base.csv"), index=False)
pd.DataFrame(X_test_base).to_csv(get_save_path("X_test_base.csv"), index=False)

pd.DataFrame(X_train_deep).to_csv(get_save_path("X_train_deep.csv"), index=False)
pd.DataFrame(X_val_deep).to_csv(get_save_path("X_val_deep.csv"), index=False)
pd.DataFrame(X_test_deep).to_csv(get_save_path("X_test_deep.csv"), index=False)

y_train.to_csv(get_save_path("y_train.csv"), index=False)
y_val.to_csv(get_save_path("y_val.csv"), index=False)
y_test.to_csv(get_save_path("y_test.csv"), index=False)

In [17]:
print(np.isnan(X_train_base).sum())  # 應該要是 0

0


In [18]:
X_train.head()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,UnansweredCalls,...,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus,InactiveSubs,HandsetDiff
41995,56.42,580.0,45.0,3.96,38.0,0.0,253.0,32.0,3.3,19.0,...,No,0,150,No,2-High,Other,Other,Unknown,0,1.0
17667,19.50,31.0,18.0,0.25,4.0,0.0,-16.0,-7.9,0.3,1.7,...,No,0,30,No,2-High,Other,Other,Unknown,0,0.0
22235,55.89,770.0,30.0,0.25,66.0,0.2,-538.0,-23.2,25.3,73.3,...,No,0,Unknown,No,2-High,Suburban,Other,Yes,0,0.0
45307,22.35,1026.0,25.0,0.00,16.0,0.0,-1026.0,-22.4,0.7,244.7,...,No,0,Unknown,No,5-Low,Suburban,Other,Unknown,0,0.0
49444,73.09,598.0,70.0,0.00,33.0,0.0,-305.0,-13.1,4.3,34.0,...,No,0,200,No,5-Low,Suburban,Other,No,1,2.0
